In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from transformers import AutoModelForSequenceClassification
from transfromers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    SchedulerType,
    default_data_collator,
    get_scheduler
)
from transformers import DataCollatorWithPadding

from sklearn.data_selection import train_test_split

### Ler's take the pretrained model 'cardiffnlp/twitter-roberta-base-sentiment' and perform its fine_tuning for our task with our train data:

In [ ]:
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"
num_labels = train_tw.target.nunique().value()

config = AutoConfig.from_pretrained(MODEL, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification(
    MODEL_NAME,
    config=config
    # ignore_mismatched_sizes=True
)

In [ ]:
train_tw, val_tw = train_test_split(train_tw, test_size=0.2, random_state=42,
                                    stratify=train_tw.target)

In [ ]:
def tokenize_function(row):
    return tokenizer(row, truncation=True)

tokenized_train = train_tw.map(tokenize_function, batched=True)
tokenized_val = val_tw.map(tokenize_function, batched=True)

### Let's introduce a function for metrics calculations:

In [ ]:
import evaluate

def metrics_calc(eval_preds):
    metric = evaluate.load('accuracy')
    logits, labels = eval_preds
    results = argmax(logits, axis=-1)
    return metric.compute(predictions=results, references=labels)

In [ ]:
from transformers import Trainer
from transformers import TrainigArguments

train_args = TrainingArguments()